In [4]:
from langchain_community.graphs import Neo4jGraph
import os
from dotenv import load_dotenv
from typing import List
from openai import AzureOpenAI
import os
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings("ignore")

**Load the environment variables from your `.env` file**

In [5]:
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

**Load the Azure openAI instance**

In [6]:
client = AzureOpenAI(
  api_key = azure_openai_api_key,  
  api_version = "2023-07-01-preview",
  azure_endpoint = azure_openai_endpoint
)

def embed_text(text:str)->List:
    """
    Embeds the given text using the specified model.

    Parameters:
        text (str): The text to be embedded.

    Returns:
        List: A list containing the embedding of the text.
    """
    response = client.embeddings.create(
    input = text,
    model= "text-embedding-ada-002"
    )
    return response.data[0].embedding

**Add Neo4j credentials (These information need to be kept secret)**

In [7]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "12345678"
NEO4J_DATABASE = 'neo4j'

In [8]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

**Sample question for RAG:**

In [9]:
question = "What movies are about love?"

**Get the questions embedding:**

In [10]:
question_embedding = embed_text(question)
question_embedding[:10]

[-0.004219826776534319,
 -0.03029683791100979,
 0.0007639749674126506,
 -0.02761838585138321,
 -0.007965870201587677,
 0.0019409307278692722,
 -0.006576106883585453,
 -0.021617135033011436,
 -0.002027790993452072,
 -0.008066943846642971]

**Perform Similarity Search using the question's embedding on the vector index of the graph database and get the results**

In [11]:
result = graph.query("""
    with $question_embedding as question_embedding      // Use the provided question embedding as 'question_embedding'
    CALL db.index.vector.queryNodes(                    // Call the vector index query function
        'movie_tagline_embeddings',                     // Name of the vector index to query against
        $top_k,                                         // Number of top results to retrieve
        question_embedding                              // The question embedding to compare against
        ) YIELD node AS movie, score                    // Yield each matched node and its similarity score
    RETURN movie.title, movie.tagline, score            // Return the title, tagline, and similarity score of each movie
    """,
    params={
        "question_embedding": question_embedding,       # Pass the question embedding as a parameter
        "top_k": 5                                      # Specify the number of top results to retrieve
    })
result

[{'movie.title': 'Heat',
  'movie.tagline': 'A Los Angeles crime saga',
  'score': 0.8910084962844849},
 {'movie.title': 'Grumpier Old Men',
  'movie.tagline': 'Still Yelling. Still Fighting. Still Ready for Love.',
  'score': 0.8878611922264099},
 {'movie.title': 'Tom and Huck',
  'movie.tagline': 'The Original Bad Boys.',
  'score': 0.8836773633956909},
 {'movie.title': 'Sense and Sensibility',
  'movie.tagline': 'Lose your heart and come to your senses.',
  'score': 0.8762467503547668},
 {'movie.title': 'Balto',
  'movie.tagline': 'Part Dog. Part Wolf. All Hero.',
  'score': 0.8755828142166138}]

**Pass the results to an LLM for the final answer**

In [12]:
prompt = f"# Question:\n{question}\n\n# Graph DB search results:\n{result}"
messages = [
    {"role": "system", "content": str(
        "You will be given the user question along with the search result of that question over a Neo4j graph database. Give the user the proper answer."
    )},
    {"role": "user", "content": prompt}
]

response = client.chat.completions.create(
    model=os.getenv("gpt_deployment_name"),
    messages=messages
)

print(response.choices[0].message.content)

Here are some movies that are about love:

1. Heat - A Los Angeles crime saga.
2. Grumpier Old Men - Still Yelling. Still Fighting. Still Ready for Love.
3. Tom and Huck - The Original Bad Boys.
4. Sense and Sensibility - Lose your heart and come to your senses.
5. Balto - Part Dog. Part Wolf. All Hero.

These movies revolve around the theme of love in different ways.


**Second example (in one go):**

In [13]:
question = "What movies are about adventure?"
question_embedding = embed_text(question)
result = graph.query("""
    with $question_embedding as question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """,
    params={
        "question_embedding": question_embedding,
        "top_k": 5
    })

prompt = f"# Question:\n{question}\n\n# Graph DB search results:\n{result}"
messages = [
    {"role": "system", "content": str(
        "You will be given the user question along with the search result of that question over a Neo4j graph database. Give the user the proper answer."
    )},
    {"role": "user", "content": prompt}
]

response = client.chat.completions.create(
    model=os.getenv("gpt_deployment_name"),
    messages=messages
)

print(response.choices[0].message.content)

The movies about adventure are:
1. Toy Story - The adventure takes off!
2. Cutthroat Island - The Course Has Been Set. There Is No Turning Back. Prepare Your Weapons. Summon Your Courage. Discover the Adventure of a Lifetime!
3. Tom and Huck - The Original Bad Boys.
4. Jumanji - Roll the dice and unleash the excitement!
